In [1]:
import easydict
import pickle
from sklearn.feature_extraction import FeatureHasher
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import time
import copy
import argparse

import os
import argparse
import numpy as np
import pickle


import time
import copy

from sklearn.feature_extraction import FeatureHasher

In [2]:
def make_dict(tokens): # make dict for every token
    doc = dict()
    
    for token in tokens:
        if token in doc:
            doc[token] += 1
        else:
            doc[token] = 1
    
    return doc

In [3]:
def make_Vocab(docs):
    word2index={'<unk>' : 0}
    for doc in docs:
        for token in doc:
            if word2index.get(token)==None:
                word2index[token]=len(word2index)
    return word2index

In [42]:
class TrainDataset(Dataset):
    
    def __init__(self):

        self.H = Hasher.transform(train_Xd)
        self.y_data = torch.from_numpy(np.array(dataDict['train_Y']))
        
    def __getitem__(self,index):
        return torch.from_numpy(self.H[index].toarray()[0]), self.y_data[index]
    
    def __len__(self):
        return self.H.shape[0]

In [43]:
class TestDataset(Dataset):
    
    def __init__(self):

        self.H = Hasher.transform(test_Xd)
        self.y_data = torch.from_numpy(np.array(dataDict['test_Y'])).type(torch.long)
        
    def __getitem__(self,index):
        return torch.from_numpy(self.H[index].toarray()[0]), self.y_data[index]
    
    def __len__(self):
        return self.H.shape[0]  

In [6]:
def split_Data(dataset,val_split,batch_size):

    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    random_seed = 42
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=0, sampler = train_sampler)
    valid_loader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=0, sampler = valid_sampler)

    dataloaders= {'train':train_loader,'val':valid_loader}

    return dataloaders

In [7]:
class TextClassifier(nn.Module):
    def __init__(self,vocab_size,hidden_dim,num_class):
        super(TextClassifier,self).__init__()
        #self.embedding = nn.EmbeddingBag(vocab_size, hidden_dim, sparse=True)
        self.input_layer = nn.Linear(vocab_size,hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, num_class)
        self.output_layer = nn.Softmax(dim=1)
        
    def forward(self,text):
        embedded = self.input_layer(text)
        output = self.hidden_layer(embedded)
        return self.output_layer(output)

In [32]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0 

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train','val']:
            if phase == 'train':
                
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0

            for inputs,labels in dataloaders[phase]:
                count += len(inputs)
                inputs,labels = inputs.to(device),labels.to(device)
                inputs,labels = Variable(inputs.float()), Variable(labels)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.type(torch.long))
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                        
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.type(torch.long).data)
                
            epoch_loss = running_loss / count
            epoch_acc = running_corrects.double() / count
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            #scheduler.step()

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Training Epoch complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [33]:
args = easydict.EasyDict({
        "hidden_dim": 10,
        "num_epoch": 1,
        "learning_rate": 1e-2,
        "batch_size": 32,
        "momentum": 0.9,
        "val_split": 0.2
})

In [34]:
lr = [1e-2, 1e-3, 1e-4]
args['learning_rate'] = lr[1]

In [35]:
args

{'hidden_dim': 10,
 'num_epoch': 1,
 'learning_rate': 0.001,
 'batch_size': 32,
 'momentum': 0.9,
 'val_split': 0.2}

In [36]:
with open("data.pickle", "rb") as f:
    dataDict = pickle.load(f)

In [37]:
W2I = make_Vocab(dataDict['train_X'])
len(W2I)

1319290

In [38]:
train_Xd = [make_dict(x) for x in dataDict['train_X']]
test_Xd = [make_dict(x) for x in dataDict['test_X']]

In [39]:
VOCAB_SIZE = min(len(W2I),10000000)
Hasher = FeatureHasher(n_features=VOCAB_SIZE).fit(train_Xd)

In [40]:
device = torch.device('cpu')
HIDDEN_DIM = args.hidden_dim
NUM_CLASS = len(set(dataDict['test_Y']))
NUM_EPOCH = args.num_epoch
LEARN_RATE = args.learning_rate
BATCH_SIZE = args.batch_size
MOMENTUM = args.momentum
VAL_SPLIT = args.val_split

In [44]:
dataloaders= split_Data(TrainDataset(),VAL_SPLIT,BATCH_SIZE)

In [45]:
model = TextClassifier(VOCAB_SIZE,HIDDEN_DIM,NUM_CLASS).to(device)
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE, momentum = MOMENTUM)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [47]:
for LEARN_RATE in [1e-2, 1e-3, 1e-4]:
    print(LEARN_RATE)
    model = TextClassifier(VOCAB_SIZE,HIDDEN_DIM,NUM_CLASS).to(device)
    criterion = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE, momentum = MOMENTUM)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    best_model = train_model(model, criterion, optimizer, scheduler, NUM_EPOCH)

0.01
Epoch 1/1
----------


KeyboardInterrupt: 